In [ ]:
from nltk.corpus import stopwords
import string
import re
def load_doc(filename):
# open the file as read only
  file = open(filename, 'r' )
# read all text
  text = file.read()
# close the file
  file.close()
  return text


In [ ]:
def clean_doc(doc):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
  tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
  stop_words = set(stopwords.words( 'english' ))
  tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
  tokens = [word for word in tokens if len(word) > 1]
  return tokens


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
filename = '/content/drive/My Drive/Colab Notebooks/Advanced_AI/review_polarity/txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)


['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anything', 'riddle', 'better', 'direct', 'f

In [ ]:
import string
import re
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
# load doc
  doc = load_doc(filename)
# clean doc
  tokens = clean_doc(doc)
# update counts
  vocab.update(tokens)
# load all docs in a directory
def process_docs(directory, vocab):
# walk through all files in the folder
  for filename in listdir(directory):
# skip any reviews in the test set
    if filename.startswith( 'cv9' ):
      continue
# create the full path of the file to open
    path = directory + '/' + filename
# add doc to vocab
    add_doc_to_vocab(path, vocab)


In [ ]:
vocab_filename = '/content/drive/My Drive/Colab Notebooks/Advanced_AI/review_polarity/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = set(vocab.split())


In [ ]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
# split into tokens by white space
  tokens = doc.split()
# prepare regex for char filtering
  re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
  tokens = [re_punc.sub( '' , w) for w in tokens]
# filter out tokens not in vocab
  tokens = [w for w in tokens if w in vocab]
  tokens = ' ' .join(tokens)
  return tokens


In [ ]:
# load all docs in a directory
def process_docs(directory, vocab, is_train):
  documents = list()
# walk through all files in the folder
  for filename in listdir(directory):
# skip any reviews in the test set
    if is_train and filename.startswith( 'cv9' ):
      continue
    if not is_train and not filename.startswith( 'cv9' ):
      continue
# create the full path of the file to open
    path = directory + '/' + filename
# load the doc
    doc = load_doc(path)
# clean doc
    tokens = clean_doc(doc, vocab)
# add to list
    documents.append(tokens)
  return documents


In [ ]:
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
def load_clean_dataset(vocab, is_train):
  neg = process_docs( '/content/drive/My Drive/Colab Notebooks/Advanced_AI/review_polarity/txt_sentoken/neg' , vocab, is_train)
  pos = process_docs( '/content/drive/My Drive/Colab Notebooks/Advanced_AI/review_polarity/txt_sentoken/pos' , vocab, is_train)
  docs = neg + pos
  labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
  return docs, labels


In [ ]:
def create_tokenizer(lines):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(lines)
  return tokenizer


In [ ]:
# max_length = max([len(s.split()) for s in train_docs])
# print( ' Maximum length: %d ' % max_length)


In [ ]:
def encode_docs(tokenizer, max_length, docs):
# integer encode
  encoded = tokenizer.texts_to_sequences(docs)
# pad sequences
  padded = pad_sequences(encoded, maxlen=max_length, padding= 'post' )
  return padded


In [ ]:
# vocab_size = len(tokenizer.word_index) + 1
# print( ' Vocabulary size: %d ' % vocab_size)


In [ ]:
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Conv1D(filters=32, kernel_size=8, activation= 'relu' ))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation= 'relu' ))
  model.add(Dense(1, activation= 'sigmoid' ))
# compile network
  model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])
# summarize defined model
  model.summary()
  plot_model(model, to_file= 'model.png' , show_shapes=True)
  return model


In [ ]:
# load the vocabulary
# vocab_filename = ' vocab.txt '
# vocab = load_doc(vocab_filename)
# vocab = set(vocab.split())
# load training data
train_docs, ytrain = load_clean_dataset(vocab, True)
# create the tokenizer



In [ ]:
# tokenizer = create_tokenizer(train_docs)
# # define vocabulary size
# vocab_size = len(tokenizer.word_index) + 1
# print( ' Vocabulary size: %d ' % vocab_size)
# # calculate the maximum sequence length
# max_length = max([len(s.split()) for s in train_docs])
# print( ' Maximum length: %d ' % max_length)
# # encode data
# Xtrain = encode_docs(tokenizer, max_length, train_docs)
# # define model
# model = define_model(vocab_size, max_length)